In [1]:
"""" Import libraries"""

import json, os
import numpy as np
import cv2
from PIL import Image
import json, os, glob
import matplotlib.pyplot as plt

In [3]:
def get_files(directory, extension):
    """
    Traverse a directory to get a list of files with a specific extension.
    
    Parameters:
    - directory: Path to the directory to search
    - extension: File extension to look for
    
    Returns:
    - files_list: List of file paths with the specified extension
    """
    files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension): # Check if file ends with the specified extension
                files_list.append(os.path.join(root, file)) # Add file to the list
    return files_list

def print_image_details(image_path):
    """
    Print the file name and size (width and height) of an image.
    
    Parameters:
    - image_path: Path to the image file
    """
    with Image.open(image_path) as img:
        print(f"{os.path.basename(image_path)} - Width: {img.width}, Height: {img.height}")

# Dictionary to map label names to class numbers
Label_Class = {'void': 0, 'nf1': 1}

def convert(database_path,file, height, width, name):
    """
    Convert annotation data to a mask image and save it.
    
    Parameters:
    - database_path: Path to the database where masks will be saved
    - file: Path to the JSON file containing annotation data
    - height: Height of the mask image
    - width: Width of the mask image
    - name: Name of the output mask image file
    """
    all_labels = json.load(open(file))  # Load JSON data from the file
    maskImage = np.zeros((height, width), dtype=np.uint8) # Create an empty mask image

    for i_class_object in all_labels['shapes']:
        i_label = i_class_object.get('label', 'void')   # Get the label, default to 'void'
        polygon = np.array(i_class_object['points'], dtype=np.int32) # Get the points of the polygon
        cv2.fillPoly(maskImage, [polygon], Label_Class[i_label]) # Fill the polygon on the mask image

    # check if the masks folder existing or not before creating new folder
    masks_saved_path = os.path.join(database_path, "masks")
    if not os.path.exists(masks_saved_path):
        os.makedirs(masks_saved_path)
    savePath = os.path.join(database_path, "masks", name)
    # Write the mask image to file
    cv2.imwrite(savePath, maskImage)

In [5]:
import os  # Importing the os module to interact with the operating system
import json  # Importing the json module to work with JSON data
import numpy as np  # Importing numpy for numerical operations
import cv2  # Importing OpenCV for image processing
from PIL import Image  # Importing the Python Imaging Library for image manipulation

# List of folder names
folders = [
    'NF100000002', 'NF100000003', 'NF100000005', 'NF100000006', 'NF100000008', 'NF100000009',
    'NF100000011', 'NF100000012', 'NF100000013', 'NF100000014', 'NF100000016', 'NF100000018', 'NF100000022'
]

# Get the current working directory
root_path = os.getcwd()  # Replace with your database path if different
print(f"Root path: {root_path}")

# Set the database path
database_path = root_path
print(f"Database path: {database_path}")

# Function to traverse a directory to get a list of files with a specific extension
def get_files(directory, extension):
    files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                files_list.append(os.path.join(root, file))
    return files_list

# Function to print the file name and size (width and height) of an image
def print_image_details(image_path):
    with Image.open(image_path) as img:
        print(f"{os.path.basename(image_path)} - Width: {img.width}, Height: {img.height}")

# Dictionary to map label names to class numbers
Label_Class = {'void': 0, 'nf1': 1}

# Function to convert annotation data to a mask image and save it
def convert(database_path, file, height, width, name):
    all_labels = json.load(open(file))
    maskImage = np.zeros((height, width), dtype=np.uint8)

    for i_class_object in all_labels['shapes']:
        i_label = i_class_object.get('label', 'void')
        polygon = np.array(i_class_object['points'], dtype=np.int32)
        cv2.fillPoly(maskImage, [polygon], Label_Class[i_label])

    masks_saved_path = os.path.join(database_path, "masks")
    if not os.path.exists(masks_saved_path):
        os.makedirs(masks_saved_path)
    savePath = os.path.join(masks_saved_path, name)
    cv2.imwrite(savePath, maskImage)

# Iterate over each folder
for folder in folders:
    folder_path = os.path.join(database_path, folder)
    print(f"Processing folder: {folder_path}")

    # Get lists of image and JSON files
    image_files = get_files(os.path.join(folder_path, 'images'), ".png")
    json_files = get_files(os.path.join(folder_path, 'jsons'), ".json")

    # Sort JSON files to match the image files
    json_files_sorted = [None] * len(image_files)
    for json_file in json_files:
        base_name = os.path.basename(json_file).replace('.json', '')
        for i, image_file in enumerate(image_files):
            if os.path.basename(image_file).replace('.png', '') == base_name:
                json_files_sorted[i] = json_file
                break

    # Filter out None values from json_files_sorted
    json_files_sorted = [file_path for file_path in json_files_sorted if file_path is not None]

    # Process each JSON file and corresponding image
    for json_file in json_files_sorted:
        base_name = os.path.basename(json_file).split(".")[0] + ".png"
        print(f"Processing file: {base_name}")

        if base_name == ".png":
            continue  # Skip if base_name is not set properly

        # Find the corresponding image file path in the image_files list
        for image_path in image_files:
            if os.path.basename(image_path) == base_name:
                # Open the image file and get its width and height
                with Image.open(image_path) as img:
                    width, height = img.size
                    print(f"Image dimensions: Width: {width}, Height: {height}")

                    # Create mask image from the annotation data and save it
                    mask_name = os.path.basename(json_file).split(".")[0] + ".jpg"
                    convert(folder_path, json_file, height, width, mask_name)
                break


Root path: c:\course22\NF1data_json
Database path: c:\course22\NF1data_json
Processing folder: c:\course22\NF1data_json\NF100000002
Processing file: 1.png
Image dimensions: Width: 1829, Height: 2167
Processing file: 2.png
Image dimensions: Width: 956, Height: 1327
Processing file: 3.png
Image dimensions: Width: 1997, Height: 638
Processing file: 4-down.png
Image dimensions: Width: 1700, Height: 755
Processing file: 4_up.png
Image dimensions: Width: 1447, Height: 490
Processing file: 5_down.png
Image dimensions: Width: 975, Height: 723
Processing file: 5_up.png
Image dimensions: Width: 978, Height: 636
Processing file: 6_up.png
Image dimensions: Width: 1666, Height: 1422
Processing file: 7_down.png
Image dimensions: Width: 602, Height: 1363
Processing file: 8.png
Image dimensions: Width: 1185, Height: 1279
Processing folder: c:\course22\NF1data_json\NF100000003
Processing file: 81_down.png
Image dimensions: Width: 1928, Height: 1298
Processing file: 82_up.png
Image dimensions: Width: 11